<a href="https://colab.research.google.com/github/harshita3002/schizoDetection/blob/main/Schizophrenia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing and setting up the Dataset from Kaggle

In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Import your kaggle.json file into the filestream before running the commands below.

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
! kaggle datasets download -d broach/button-tone-sz
! kaggle datasets download -d broach/buttontonesz2

100% 8.03G/8.03G [01:23<00:00, 98.4MB/s]
100% 8.03G/8.03G [01:23<00:00, 103MB/s] 
100% 8.09G/8.11G [01:23<00:00, 149MB/s]
100% 8.11G/8.11G [01:23<00:00, 105MB/s]


In [ ]:
import os
import time
import numpy as np
import pandas as pd

import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

import warnings
warnings.filterwarnings('ignore')

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

! for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done

tee: '/sys/bus/pci/devices/0000:00:00.0/numa_node': Read-only file system
0
tee: '/sys/bus/pci/devices/0000:00:01.0/numa_node': Read-only file system
0
tee: '/sys/bus/pci/devices/0000:00:01.3/numa_node': Read-only file system
0
tee: '/sys/bus/pci/devices/0000:00:03.0/numa_node': Read-only file system
0
tee: '/sys/bus/pci/devices/0000:00:04.0/numa_node': Read-only file system
0
tee: '/sys/bus/pci/devices/0000:00:05.0/numa_node': Read-only file system
0
tee: '/sys/bus/pci/devices/0000:00:06.0/numa_node': Read-only file system
0


In [ ]:
! cd button-tone-sz/
! unzip button-tone-sz.zip -d button-tone-sz/
! unzip buttontonesz2.zip -d button-tone-sz/

/bin/bash: line 1: cd: button-tone-sz/: No such file or directory
Archive:  button-tone-sz.zip
  inflating: button-tone-sz/1.csv/1.csv  
  inflating: button-tone-sz/10.csv/10.csv  
  inflating: button-tone-sz/11.csv/11.csv  
  inflating: button-tone-sz/12.csv/12.csv  
  inflating: button-tone-sz/13.csv/13.csv  
  inflating: button-tone-sz/14.csv/14.csv  
  inflating: button-tone-sz/15.csv/15.csv  
  inflating: button-tone-sz/16.csv/16.csv  
  inflating: button-tone-sz/17.csv/17.csv  
  inflating: button-tone-sz/18.csv/18.csv  
  inflating: button-tone-sz/19.csv/19.csv  
  inflating: button-tone-sz/2.csv/2.csv  
  inflating: button-tone-sz/20.csv/20.csv  
  inflating: button-tone-sz/21.csv/21.csv  
  inflating: button-tone-sz/22.csv/22.csv  
  inflating: button-tone-sz/23.csv/23.csv  
  inflating: button-tone-sz/24.csv/24.csv  
  inflating: button-tone-sz/3.csv/3.csv  
  inflating: button-tone-sz/4.csv/4.csv  
  inflating: button-tone-sz/5.csv/5.csv  
  inflating: button-tone-sz/6.csv/6

Reading CSV Data


In [ ]:
demographic = pd.read_csv("/content/button-tone-sz/demographic.csv")
demographic

,subject,group,gender,age,education
0,1,0,M,44,16.0
1,2,0,M,39,17.0
2,3,0,M,53,18.0
3,4,0,M,52,15.0
4,5,0,M,41,16.0
...,...,...,...,...,...
76,77,1,M,28,13.0
77,78,1,F,32,16.0
78,79,1,M,37,16.0
79,80,1,M,33,13.0


In [ ]:
columns = pd.read_csv("/content/button-tone-sz/time.csv")
columns

,sample,time_ms
0,1,-1500.0000
1,2,-1499.0234
2,3,-1498.0469
3,4,-1497.0703
4,5,-1496.0938
...,...,...
3067,3068,1495.1172
3068,3069,1496.0938
3069,3070,1497.0703
3070,3071,1498.0469


In [ ]:
diagnosis_dict = dict(zip(demographic.subject, demographic[" group"]))
electrodes_list = list(pd.read_csv("/content/button-tone-sz/columnLabels.csv").columns[4:])
print("Electrodes List \n",electrodes_list)

Electrodes List 
 ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'VEOa', 'VEOb', 'HEOL', 'HEOR', 'Nose', 'TP10']


In [ ]:
def averaged_by_N_rows(a, n):
    """
    A function that averages n rows in a matrix
    """
    shape = a.shape
    assert len(shape) == 2
    assert shape[0] % n == 0
    b = a.reshape(shape[0] // n, n,
                  shape[1])
    mean_vec = b.mean(axis=1)
    return mean_vec

N_AVERAGED = 16
x_counter = 0
X = np.zeros((81 * 100,  9216 * len(electrodes_list) // N_AVERAGED), dtype="float32")
Y = np.zeros(len(X))

part1_path = "../content/button-tone-sz"
part2_path = "../content/button-tone-sz"


column_list = pd.read_csv("/content/button-tone-sz/columnLabels.csv").columns
for person_number in tqdm(range(1, 81 + 1)):


    csv_path = f"{part1_path}/{person_number}.csv/{person_number}.csv"
    if not os.path.exists(csv_path):
        csv_path = f"{part2_path}/{person_number}.csv/{person_number}.csv"
    df = pd.read_csv(csv_path,
                header=None,
                names=column_list)
    trials_list = set(df.trial)


    for t1, trial_number in enumerate(trials_list):
        number_of_trials = len(df[df.trial == trial_number])
        if number_of_trials == 9216.0:
            current_sample_matrix = df[df.trial == trial_number][electrodes_list].values
            averaged_by_N = averaged_by_N_rows(current_sample_matrix, n=N_AVERAGED)
            averaged_by_N_big_vec = averaged_by_N.reshape(-1)
            X[x_counter] = averaged_by_N_big_vec.astype(np.float32)
            Y[x_counter] = diagnosis_dict[person_number]
            x_counter += 1
print("Total trials with the Appropriate number of measurements - ", x_counter)
X = X[: x_counter]
Y = Y[: x_counter]

print("Total trials with the Appropriate number of measurements- ", x_counter)

  0%|          | 0/81 [00:00<?, ?it/s]

Total trials with the Appropriate number of measurements -  7092
Total trials with the Appropriate number of measurements-  7092


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)
X_norm = (normalize(X.reshape(-1, 70), axis=0, norm='max')).reshape(X.shape)
X_train_norm, X_test_norm, Y_train_norm, Y_test_norm = train_test_split(X_norm, Y, test_size=0.2, shuffle=True, random_state=42)
_norm = X
X_train_2d = X_train_norm.reshape(X_train_norm.shape[0], len(electrodes_list), X_train_norm.shape[1] // len(electrodes_list), 1)
X_test_2d = X_test_norm.reshape(X_test_norm.shape[0], len(electrodes_list), X_test_norm.shape[1] // len(electrodes_list), 1)

In [ ]:
# filepath="/kaggle/working/best_model1.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 20),
                 activation='tanh',
                 input_shape=(X_train_2d.shape[1:])))
model.add(MaxPooling2D(pool_size=(5, 15)))

model.add(Conv2D(13, kernel_size=(3, 3),
                 activation='tanh',))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(317, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tf.optimizers.Adam(0.0000075),
              metrics=['acc'])
history_params = []

Training the Model

In [ ]:
history = model.fit(X_train_2d, Y_train_norm,
          batch_size=256,
          epochs=10,
          verbose=1,
          shuffle=True,
          validation_data=(X_test_2d, Y_test_norm))

Epoch 1/10
23/23 [==============================] - 186s 8s/step - loss: 0.6925 - acc: 0.5489 - val_loss: 0.6920 - val_acc: 0.5821
Epoch 2/10
23/23 [==============================] - 186s 8s/step - loss: 0.6911 - acc: 0.5912 - val_loss: 0.6906 - val_acc: 0.5920
Epoch 3/10
23/23 [==============================] - 183s 8s/step - loss: 0.6895 - acc: 0.5983 - val_loss: 0.6892 - val_acc: 0.5927
Epoch 4/10
23/23 [==============================] - 183s 8s/step - loss: 0.6879 - acc: 0.5979 - val_loss: 0.6876 - val_acc: 0.5927
Epoch 5/10
23/23 [==============================] - 187s 8s/step - loss: 0.6863 - acc: 0.5983 - val_loss: 0.6860 - val_acc: 0.5927
Epoch 6/10
23/23 [==============================] - 182s 8s/step - loss: 0.6845 - acc: 0.5983 - val_loss: 0.6842 - val_acc: 0.5927
Epoch 7/10
23/23 [==============================] - 183s 8s/step - loss: 0.6826 - acc: 0.5983 - val_loss: 0.6824 - val_acc: 0.5927
Epoch 8/10
23/23 [==============================] - 183s 8s/step - loss: 0.6805 - a